## Getting Long-Term Daily Google Trends Data

### Template

In [1]:
# importing dependencies
import pytrends
from pytrends.request import TrendReq
import pandas as pd
from datetime import date, datetime, timedelta

In [2]:
# Specify the parameters to your liking
start_date= date(2015, 1, 1)
end_date= date(2019, 1, 1)
key_word = '' # use one key word
_cat = 0
_geo = ''
_gprop = ''
_hl = 'en-US'
_tz = 360

In [4]:
# Source for this function: https://stackoverflow.com/questions/10688006/generate-a-list-of-datetimes-between-an-interval
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta

In [5]:
# generating a list of dates with 90 day intervals
dates=[]
for res in perdelta(start_date, end_date, timedelta(days=90)):
    dates.append(res)  
dates.append(end_date)

In [6]:
# gettingt the data for the individual time frames and adding them to a list
appended_data = []
for i in range(len(dates)-1):
    _timeframe = str(dates[i]) + ' ' + str(dates[i+1])
    totalTrend = TrendReq(hl=_hl, tz=_tz)
    totalTrend.build_payload([key_word], cat=_cat, timeframe=_timeframe, geo=_geo, gprop=_gprop)
    totalTrend = totalTrend.interest_over_time()
    appended_data.append(totalTrend)
#   print(totalTrend.tail())

In [7]:
# correcting the values of the individual time-frames
for i in  range(len(appended_data)-1):
    x = appended_data[i][key_word].tail(1)
    y = appended_data[i+1][key_word].head(1)
    factor = float(x/y)
    appended_data[i+1][key_word] = appended_data[i+1][key_word] * factor

In [8]:
# concatinating all the dfs to one complete dataframe
appended_df = pd.concat(appended_data, axis=0)

In [9]:
# deleting the duplicated indexes/rows
appended_df = appended_df[~appended_df.index.duplicated(keep='first')]

In [10]:
# saving the daily data as a csv
appended_df.to_csv('daily_gtrends.csv')